# Decanter AI Core SDK Jupyter Notebook Example
This example provides how to use the python package decanter-ai-core-sdk, including installation, usage of the apis, and some ways to show chart of the experiment's attributes and data values. 

## Installation

In [ ]:
!pip install decanter-ai-core-sdk

## Running Example

In [ ]:
import os
from decanter import core
from decanter.core.core_api import TrainInput, PredictInput
from decanter.core.enums.algorithms import Algos as Alg
from decanter.core.enums.evaluators import Evaluator as Eva

### Create Context
Create Context will set the connection to decanter core server, and create an event loop. Since Jupyter already have an event loop, SDK will just use the current event loop. See more in [here](https://www.notion.so/API-615d2fba4e7f45c4b5fe63cc192e481f#bb4f0a4b2847450abc4f80b025469170)

In Jupyter, it will initially exist a running event loop.

In [ ]:
import asyncio
loop = asyncio.get_running_loop()
loop.is_running()

Note: It is ok to recreate context to set the usr, pwd, host.

In [ ]:
# enable default logger
core.enable_default_logger()
# set the username, password, host
context = core.Context.create(
        username='{usr}', password='{pwd}', host='{decantercoreserver}')

### Create Decanter Core Client
CoreClient handles the actions of calling api and getting the results.

In [ ]:
client = core.CoreClient()

Example of using `core.upload`, `core.train`, `core.predict`.
While it's running, it'll show up the progress bar showing the progress of current running jobs.

In [ ]:
# open train & test file
train_file_path = os.path.join('{train_file_path}')
test_file_path = os.path.join('{test_file_path}')
train_file = open(train_file_path , 'r')
test_file = open(test_file_path , 'r')

# upload data to corex 
train_data = client.upload(file=train_file, name="train_data")
test_data = client.upload(file=test_file, name="test_data")
# from decanter.core.jobs import DataUpload
# train_data = DataUpload.create(data_id = "{data_id}", name="train_data")

# set train parameters train model
train_input = TrainInput(data=train_data, target='Survived', algos=[Alg.XGBoost], max_model=2, tolerance=0.9)
exp = client.train(train_input=train_input, select_model_by=Eva.mean_per_class_error, name='myexp')

# set predict parameters and predict result
predict_input = PredictInput(data=test_data, experiment=exp)
pred_res = client.predict(predict_input=predict_input, name='mypred')

In [ ]:
# To prevent getting attributes when corresponding jobs aren't finished
exp.get(attr='attributes')
exp.best_model.get(attr='importances')
pred_res.get(attr='schema')

### Show Data
Use `data.show_df()` and `predict_result.show_df()` to create pandas dataframe. <br>
Use `data.show()` or `predict_result.show()` to show data in text.

In [ ]:
train_data.show_df()

### Show models attributes
List the attributes you wish to see in a list, and call `core.plot.show_model_attr(attrs, exp)`

In [ ]:
core.plot.show_model_attr(metric=Eva.mean_per_class_error, score_types=['validation', 'cv_averages'], exp=exp)

### Get Data values with Data instance
Plot chart with the values in data instance

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


labels = ['Survied', 'Dead']
labels_gender = ['male', 'female', 'male', 'female']


df = train_data.show_df()
df_s = df.loc[df["Survived"] == 1]
df_d = df.loc[df["Survived"] == 0]

sizes = [df_s.shape[0], df_d.shape[0]]

sizes_gender = []
for d in [df_s, df_d]:
    s1 = d.loc[d["Sex"] == "male"].shape[0]
    s2 = d.loc[d["Sex"] == "female"].shape[0]
    sizes_gender.append(s1)
    sizes_gender.append(s2)

    
colors = ['#99ff99', '#ff6666']
colors_gender = ['#87CEFA','#FFB6C1', '#87CEFA','#FFB6C1']
 
# Plot
plt.pie(sizes, labels=labels, colors=colors, startangle=90,frame=True)
plt.pie(sizes_gender,colors=colors_gender ,radius=0.75,startangle=90)
centre_circle = plt.Circle((0,0),0.5,color='black', fc='white',linewidth=0)
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
 
plt.axis('equal')
plt.tight_layout()
plt.show()